In [95]:
import numpy as np
import pandas as pd

In [105]:
data = pd.DataFrame({
    "id": [1,1,1,1,2,2,2,2,3,3],
    "valid": [1,1,0,1,1,0,1,0,1,1],
    "delta": [1,2,3,4,5,6,7,8,9,10]
})

num_it = int(data.groupby("id").agg({"id": "count"}).max())


In [106]:
data

,id,valid,delta
0,1,1,1
1,1,1,2
2,1,0,3
3,1,1,4
4,2,1,5
5,2,0,6
6,2,1,7
7,2,0,8
8,3,1,9
9,3,1,10


In [109]:
import numpy as np
import pandas as pd

data = pd.DataFrame({
    "id": [1,1,1,1,2,2,2,2,3,3],
    "valid": [1,1,0,1,1,0,1,0,1,1],
    "delta": [1,2,3,4,5,6,7,8,9,10]
})

num_it = int(data.groupby("id").agg({"id": "count"}).max())

data["delta_agg"] = 0
data["delta_select"] = data["delta"] * data["valid"]

for i in range(num_it):

    data[["id_t", "delta_t"]] = np.roll(data[["id", "delta_select"]], i, axis = 0)
    data.loc[(data["id_t"] == data["id"]), "delta_agg"] += data.loc[(data["id_t"] == data["id"]), "delta_t"]
    
data.drop(["id_t", "delta_t", "delta_select"], axis = 1, inplace = True)

In [110]:
data

,id,valid,delta,delta_agg
0,1,1,1,1
1,1,1,2,3
2,1,0,3,3
3,1,1,4,7
4,2,1,5,5
5,2,0,6,5
6,2,1,7,12
7,2,0,8,12
8,3,1,9,9
9,3,1,10,19


In [1]:
def aggregate_timeseries_by_id(df, id_col, data_col):
    """
    Aggregates the data_col accross time series, split by account id.
    This function assumes that the df is sorted by date and id.
    i.e. blocks of the same id sorted from t = 0 -> t = T.
    """
    # Calculating the maximum number of iterations required to iterate accross
    # the entirety of the most frequent account id.
    num_it = int(df.groupby(id_col).agg({id_col: "count"}).max())
    
    # Initialising the aggregate data column to zero
    df[f"{data_col}_aggregate"] = 0
    
    for roll in range(num_it):
        # Roll forward the id and data columns in df by the designated roll number
        # and store them as temporary columns
        df[[f"{id_col}_temp", f"{data_col}_temp"]] = np.roll(df[[id_col, data_col]], roll, axis = 0)
        
        # Where the temporary id is the same as the row id, add the rolled temporary
        # data column to the aggregate data column
        df.loc[
            (df[f"{id_col}_temp"] == df[id_col]),
            f"{data_col}_aggregate"
        ] += df.loc[
            (df[f"{id_col}_temp"] == df[id_col]),
            f"{data_col}_temp"
        ]
    
    # Drop the temporary columns after final iteration
    df.drop([f"{id_col}_temp", f"{data_col}_temp"], axis = 1, inplace = True)
    
    return df

In [2]:
import numpy as np
import pandas as pd

data = pd.DataFrame({
    "id": [1,1,1,1,2,2,2,2,3,3],
    "valid": [1,1,0,1,1,0,1,0,1,1],
    "delta": [1,2,3,4,5,6,7,8,9,10]
})

data["delta_select"] = data["delta"] * data["valid"]

data = aggregate_timeseries_by_id(data, "id", "delta_select")

,id,valid,delta,delta_select,delta_select_aggregate
0,1,1,1,1,1
1,1,1,2,2,3
2,1,0,3,0,3
3,1,1,4,4,7
4,2,1,5,5,5
5,2,0,6,0,5
6,2,1,7,7,12
7,2,0,8,0,12
8,3,1,9,9,9
9,3,1,10,10,19
